In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

In [17]:
# Load dữ liệu
red = pd.read_csv("Red.csv")
white = pd.read_csv("White.csv")
rose = pd.read_csv("Rose.csv")
sparkling = pd.read_csv("Sparkling.csv")

# Gắn nhãn và gộp
red["WineStyle"] = "Red"
white["WineStyle"] = "White"
rose["WineStyle"] = "Rose"
sparkling["WineStyle"] = "Sparkling"
df = pd.concat([red, white, rose, sparkling], ignore_index=True)

# Xử lý dữ liệu
df = df[df["Year"] != "N.V."].copy()
df["Year"] = df["Year"].astype(int)
df["Age"] = 2025 - df["Year"]
df["Variety"] = "Unknown"
df["PricePerAge"] = df["Price"] / (df["Age"] + 1)
df["IsOld"] = df["Age"] >= 10
df["LogPrice"] = np.log1p(df["Price"])
df["PriceAgeInteraction"] = df["PricePerAge"] * df["LogPrice"]
df["LogLogPrice"] = np.log1p(df["LogPrice"])

# SmoothedRating
global_mean = df["Rating"].mean()
df["SmoothedRating"] = (df["Rating"] * df["NumberOfRatings"] + global_mean * 5) / (df["NumberOfRatings"] + 5)

# Reliability Group
def reliability_group(n):
    if n < 35: return "Low"
    elif n < 838: return "Medium"
    else: return "High"
df["ReliabilityGroup"] = df["NumberOfRatings"].apply(reliability_group)

# Đặc trưng nâng cao
df["Country_Rank"] = df["Country"].map(df.groupby("Country")["SmoothedRating"].mean().rank(ascending=False, method="min").astype(int))
df["Region_Rank"] = df["Region"].map(df.groupby("Region")["SmoothedRating"].mean().rank(ascending=False, method="min").astype(int))
winery_mean = df.groupby("Winery")["SmoothedRating"].mean()
df["Winery_Score"] = df["Winery"].map((winery_mean.max() - winery_mean) / (winery_mean.max() - 1))
df["Variety_Freq"] = df["Variety"].map(df["Variety"].value_counts(normalize=True))


In [18]:
features = ["Country", "Region", "Winery", "Price", "WineStyle", "Variety", "Age",
            "NumberOfRatings", "PricePerAge", "IsOld", "LogPrice", "PriceAgeInteraction",
            "LogLogPrice", "Country_Rank", "Region_Rank", "Winery_Score", "Variety_Freq"]
target = "SmoothedRating"

# Chia nhóm
low_df = df[df["ReliabilityGroup"] == "Low"].dropna(subset=features + [target])
mid_df = df[df["ReliabilityGroup"] == "Medium"].dropna(subset=features + [target])
high_df = df[df["ReliabilityGroup"] == "High"].dropna(subset=features + [target])

# Chia train/test
low_train, low_test = train_test_split(low_df, test_size=0.2, random_state=42)
mid_train, mid_test = train_test_split(mid_df, test_size=0.2, random_state=42)
high_train, high_test = train_test_split(high_df, test_size=0.2, random_state=42)

# Blend thêm 30% Mid vào Low & High
mid_sample_for_low = mid_train.sample(frac=0.5, random_state=1)
mid_sample_for_high = mid_train.sample(frac=0.5, random_state=2)
low_train_blended = pd.concat([low_train, mid_sample_for_low], ignore_index=True)
high_train_blended = pd.concat([high_train, mid_sample_for_high], ignore_index=True)


In [19]:
import os
from autogluon.tabular import TabularPredictor

# 1. Khởi tạo: tạo thư mục models
os.makedirs("models", exist_ok=True)

# 2. Thiết lập biến dùng chung
TIME_LIMIT = 300            # 300 giây = 5 phút
PRESETS = 'medium_quality_faster_train'
# Đảm bảo bạn đã định nghĩa mid_train, low_train_blended, high_train_blended, features, target

# 3. Huấn luyện & save nhóm Medium
print("🚀 Bắt đầu huấn luyện AutoGluon – Medium …")
predictor_mid = TabularPredictor(label=target).fit(
    train_data=mid_train[features + [target]],
    time_limit=TIME_LIMIT,
    presets=PRESETS
)
print("✅ Đã xong fit Medium. Chuẩn bị save …")
predictor_mid.save("models/AutoGluon_Mid")
print("✅ Đã save AutoGluon_Mid")

# 4. Huấn luyện & save nhóm Low
print("🚀 Bắt đầu huấn luyện AutoGluon – Low …")
predictor_low = TabularPredictor(label=target).fit(
    train_data=low_train_blended[features + [target]],
    time_limit=TIME_LIMIT,
    presets=PRESETS
)
print("✅ Đã xong fit Low. Chuẩn bị save …")
predictor_low.save("models/AutoGluon_Low")
print("✅ Đã save AutoGluon_Low")

# 5. Huấn luyện & save nhóm High
print("🚀 Bắt đầu huấn luyện AutoGluon – High …")
predictor_high = TabularPredictor(label=target).fit(
    train_data=high_train_blended[features + [target]],
    time_limit=TIME_LIMIT,
    presets=PRESETS
)
print("✅ Đã xong fit High. Chuẩn bị save …")
predictor_high.save("models/AutoGluon_High")
print("✅ Đã save AutoGluon_High")

print("🎉 Hoàn tất huấn luyện và lưu cả 3 nhóm AutoGluon.")


No path specified. Models will be saved in: "AutogluonModels\ag-20250606_112729"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       8.35 GB / 15.36 GB (54.4%)
Disk Space Avail:   7.54 GB / 201.51 GB (3.7%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.
Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "c:\Users\Admin\Downloads\Hieu\AutogluonModels\ag-20250606_112729"
Train Data Rows:    8497
Train Data Columns: 17
Label Column:       SmoothedRating
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values

🚀 Bắt đầu huấn luyện AutoGluon – Medium …


	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: KNeighborsDist ... Training model for up to 299.84s of the 299.84s of remaining time.
	-0.1984	 = Validation score   (-root_mean_squared_error)
	0.0s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ... Training model for up to 299.81s of the 299.80s of remaining time.
	-0.147	 = Validation score   (-root_mean_squared_error)
	0.85s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 298.92s of the 298.92s of remaining time.
	-0.1447	 = Validation score   (-root_mean_squared_error)
	0.65s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 298.24s of the 298.24s of remaining time.
	-0.1483	 = Validation score   (-root_mean_squared_error)
	2.98s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 295.05s of the 295.05s of remain

✅ Đã xong fit Medium. Chuẩn bị save …
✅ Đã save AutoGluon_Mid
🚀 Bắt đầu huấn luyện AutoGluon – Low …


	0.0s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ... Training model for up to 299.82s of the 299.82s of remaining time.
	-0.1576	 = Validation score   (-root_mean_squared_error)
	0.69s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 299.12s of the 299.12s of remaining time.
	-0.156	 = Validation score   (-root_mean_squared_error)
	0.65s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 298.44s of the 298.44s of remaining time.
	-0.1588	 = Validation score   (-root_mean_squared_error)
	1.95s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 296.35s of the 296.35s of remaining time.
	-0.1502	 = Validation score   (-root_mean_squared_error)
	31.57s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 264.76s of the 264.76s of remain

✅ Đã xong fit Low. Chuẩn bị save …
✅ Đã save AutoGluon_Low
🚀 Bắt đầu huấn luyện AutoGluon – High …


	-0.205	 = Validation score   (-root_mean_squared_error)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ... Training model for up to 299.83s of the 299.83s of remaining time.
	-0.1438	 = Validation score   (-root_mean_squared_error)
	0.8s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 298.99s of the 298.99s of remaining time.
	-0.1454	 = Validation score   (-root_mean_squared_error)
	0.57s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 298.41s of the 298.40s of remaining time.
	-0.1486	 = Validation score   (-root_mean_squared_error)
	1.85s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 296.42s of the 296.42s of remaining time.
	-0.1419	 = Validation score   (-root_mean_squared_error)
	37.41s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE ... 

✅ Đã xong fit High. Chuẩn bị save …
✅ Đã save AutoGluon_High
🎉 Hoàn tất huấn luyện và lưu cả 3 nhóm AutoGluon.


In [21]:
import os
print("Working directory right now:", os.getcwd())
print("Các file/folder trong đó:", os.listdir(os.getcwd()))


Working directory right now: c:\Users\Admin\Downloads\Hieu
Các file/folder trong đó: ['.vscode', 'AutogluonModels', 'AutoGluon_Main.ipynb', 'CatBoostMain - Copy.ipynb', 'catboost_info', 'ExtraTrees_ByGroup.ipynb', 'HistGradientBoosting_ByGroup.ipynb', 'LightGBM_ByGroup.ipynb', 'models', 'RandomForest_ByGroup.ipynb', 'Red.csv', 'Rose.csv', 'Sparkling.csv', 'Varieties.csv', 'White.csv', 'XGBoostEncoded.ipynb']


In [22]:
import os, shutil

# Bước 1: Chuyển tới đúng working directory
os.chdir(r"c:\Users\Admin\Downloads\Hieu")

# Bước 2: Đảm bảo thư mục models tồn tại
os.makedirs("models", exist_ok=True)

# Bước 3: Đặt đường dẫn gốc AutoGluon của mỗi nhóm theo log
src_low  = r"AutogluonModels\ag-20250606_112944"
src_mid  = r"AutogluonModels\ag-20250606_112854"
src_high = r"AutogluonModels\ag-20250606_112729"

# Bước 4: Thư mục đích trong models/
dst_low  = r"models\AutoGluon_Low"
dst_mid  = r"models\AutoGluon_Mid"
dst_high = r"models\AutoGluon_High"

# Bước 5: Copy nguyên thư mục vào models/ với dirs_exist_ok=True để ghi đè nếu đã tồn tại
shutil.copytree(src_low,  dst_low,  dirs_exist_ok=True)
shutil.copytree(src_mid,  dst_mid,  dirs_exist_ok=True)
shutil.copytree(src_high, dst_high, dirs_exist_ok=True)

# Bước 6 (tùy chọn): In ra nội dung để kiểm tra
print("models/:", os.listdir("models"))
print("AutoGluon_Low contents:", os.listdir(dst_low))
print("AutoGluon_Mid contents:", os.listdir(dst_mid))
print("AutoGluon_High contents:", os.listdir(dst_high))


models/: ['AutoGluon_High', 'AutoGluon_Low', 'AutoGluon_Mid', 'CatBoost_High.pkl', 'CatBoost_Low.pkl', 'CatBoost_Mid.pkl', 'ExtraTrees_High.pkl', 'ExtraTrees_Low.pkl', 'ExtraTrees_Medium.pkl', 'HistGB_High.pkl', 'HistGB_Low.pkl', 'HistGB_Medium.pkl', 'LightGBM_High.txt', 'LightGBM_Low.txt', 'LightGBM_Medium.txt', 'RandomForest_High.pkl', 'RandomForest_Low.pkl', 'RandomForest_Medium.pkl', 'XGBoost_Encoders_High.pkl', 'XGBoost_Encoders_Low.pkl', 'XGBoost_Encoders_Mid.pkl', 'XGBoost_High.pkl', 'XGBoost_Low.pkl', 'XGBoost_Mid.pkl']
AutoGluon_Low contents: ['learner.pkl', 'metadata.json', 'models', 'predictor.pkl', 'utils', 'version.txt']
AutoGluon_Mid contents: ['learner.pkl', 'metadata.json', 'models', 'predictor.pkl', 'utils', 'version.txt']
AutoGluon_High contents: ['learner.pkl', 'metadata.json', 'models', 'predictor.pkl', 'utils', 'version.txt']


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mae, rmse, r2, mape

print("📊 Kết quả trên tập TEST:")

# Nhóm Low
y_low_test = low_test[target]
y_low_pred = predictor_low.predict(low_test[features])    # dùng predictor_low
mae, rmse, r2, mape = evaluate(y_low_test, y_low_pred)
print(f"Low     — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

# Nhóm Medium
y_mid_test = mid_test[target]
y_mid_pred = predictor_mid.predict(mid_test[features])    # dùng predictor_mid
mae, rmse, r2, mape = evaluate(y_mid_test, y_mid_pred)
print(f"Medium  — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

# Nhóm High
y_high_test = high_test[target]
y_high_pred = predictor_high.predict(high_test[features])  # dùng predictor_high
mae, rmse, r2, mape = evaluate(y_high_test, y_high_pred)
print(f"High    — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

print("\n🧪 Kết quả trên tập TRAIN:")

# Nhóm Low (train)
y_low_train = low_train_blended[target]
y_low_pred_tr = predictor_low.predict(low_train_blended[features])
mae, rmse, r2, mape = evaluate(y_low_train, y_low_pred_tr)
print(f"Low     Train — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

# Nhóm Medium (train)
y_mid_train = mid_train[target]
y_mid_pred_tr = predictor_mid.predict(mid_train[features])
mae, rmse, r2, mape = evaluate(y_mid_train, y_mid_pred_tr)
print(f"Medium  Train — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

# Nhóm High (train)
y_high_train = high_train_blended[target]
y_high_pred_tr = predictor_high.predict(high_train_blended[features])
mae, rmse, r2, mape = evaluate(y_high_train, y_high_pred_tr)
print(f"High    Train — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")


📊 Kết quả trên tập TEST:
Low     — MAE: 0.1114, RMSE: 0.1442, R²: 0.7170, MAPE: 2.94%
Medium  — MAE: 0.1063, RMSE: 0.1392, R²: 0.7499, MAPE: 2.77%
High    — MAE: 0.0962, RMSE: 0.1236, R²: 0.8160, MAPE: 2.44%

🧪 Kết quả trên tập TRAIN:
Low     Train — MAE: 0.0890, RMSE: 0.1180, R²: 0.8231, MAPE: 2.33%
Medium  Train — MAE: 0.0868, RMSE: 0.1152, R²: 0.8330, MAPE: 2.27%
High    Train — MAE: 0.0804, RMSE: 0.1075, R²: 0.8592, MAPE: 2.10%
